In [2]:
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
import xgboost as xgb

In [1]:
train_transaction = pd.read_csv('E:\\Projects folder\\ieee-fraud-detection\\train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('E:\\Projects folder\\ieee-fraud-detection\\test_transaction.csv', index_col='TransactionID')
train_identity = pd.read_csv('E:\\Projects folder\\ieee-fraud-detection\\train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('E:\\Projects folder\\ieee-fraud-detection\\test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('E:\\Projects folder\\ieee-fraud-detection\\sample_submission.csv', index_col='TransactionID')

NameError: name 'pd' is not defined

In [5]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)


In [8]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 2987000 to 3577539
Columns: 433 entries, isFraud to DeviceInfo
dtypes: float64(399), int64(3), object(31)
memory usage: 1.9+ GB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 3663549 to 4170239
Columns: 432 entries, TransactionDT to DeviceInfo
dtypes: float64(399), int64(2), object(31)
memory usage: 1.7+ GB


In [9]:
pd.isnull(train).any()

isFraud           False
TransactionDT     False
TransactionAmt    False
ProductCD         False
card1             False
card2              True
card3              True
card4              True
card5              True
card6              True
addr1              True
addr2              True
dist1              True
dist2              True
P_emaildomain      True
R_emaildomain      True
C1                False
C2                False
C3                False
C4                False
C5                False
C6                False
C7                False
C8                False
C9                False
C10               False
C11               False
C12               False
C13               False
C14               False
                  ...  
id_11              True
id_12              True
id_13              True
id_14              True
id_15              True
id_16              True
id_17              True
id_18              True
id_19              True
id_20              True
id_21           

In [10]:
y_train = train['isFraud'].copy()
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [11]:
pd.isnull(X_train).any()

TransactionDT     False
TransactionAmt    False
ProductCD         False
card1             False
card2             False
card3             False
card4             False
card5             False
card6             False
addr1             False
addr2             False
dist1             False
dist2             False
P_emaildomain     False
R_emaildomain     False
C1                False
C2                False
C3                False
C4                False
C5                False
C6                False
C7                False
C8                False
C9                False
C10               False
C11               False
C12               False
C13               False
C14               False
D1                False
                  ...  
id_11             False
id_12             False
id_13             False
id_14             False
id_15             False
id_16             False
id_17             False
id_18             False
id_19             False
id_20             False
id_21           

In [12]:
del train, test, train_transaction, train_identity, test_transaction, test_identity

In [13]:
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

In [14]:
clf = xgb.XGBClassifier(n_estimators=500,
                        n_jobs=4,
                        max_depth=9,
                        learning_rate=0.05,
                        subsample=0.9,
                        colsample_bytree=0.9,
                        missing=-999)

clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=-999, n_estimators=500, n_jobs=4,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.9, verbosity=1)

In [1]:
sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.to_csv('E:\\Projects folder\\ieee-fraud-detection\\simple_xgboost.csv')

NameError: name 'clf' is not defined